In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
%%time
# all_data = pd.read_csv('./data/task1_all_data.tsv', sep='\t')
all_data = pd.read_csv('./data/cnews.train.txt', sep='\t')

x, y = all_data['Phrase'], all_data['Sentiment']
x_train1, x_val, y_train1, y_val = train_test_split(x, y, test_size=0.2)
x_train, x_test, y_train, y_test = train_test_split(x_train1, y_train1, test_size=0.125)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

Wall time: 201 ms


In [3]:
%%time
count_vector = CountVectorizer(max_features= 1000)
count_vector.fit_transform(x_train)
x_train = count_vector.transform(x_train).toarray()
x_val= count_vector.transform(x_val).toarray()

Wall time: 2.46 s


In [4]:
class LogisticRegressionClassifier:
    def __init__(self, max_iter=200, lr=0.01):
        self.max_iter = max_iter
        self.lr = lr

    def sigmoid(self, X):
        return 1/(1+np.exp(-X)) 
    
    def softmax(self, d):
        return np.exp(d) / np.sum(np.exp(d))
    
    def data_matrix(self, X):
        data_mat = []
        for x in X:
            data_mat.append([1, *x])
        return data_mat
    
    def fit(self, X, y):
        data_mat = self.data_matrix(X)
        self.weights = np.zeros((len(data_mat[0]),5), dtype = np.float32) 

        for step_ in range(self.max_iter):
            for i in range(len(data_mat)):
                # pre = self.sigmoid(np.dot(data_mat[i], self.weights))
                pre = self.softmax(np.dot(data_mat[i], self.weights))
                obj = np.eye(5)[int(y[i])]
                err = pre - obj
                self.weights -= self.lr * np.transpose([data_mat[i]]) * err
            if(step_%10==0):
                print("*********************************************************")
                print("round {}\n \nerr {} \nscore {}".format(step_, err, self.score(x_val,y_val)))

    def score(self, X, y):
        X=self.data_matrix(X)
        right = 0
        for i in range(len(X)):
            pre = np.dot(X[i], self.weights)
            pre2 = np.argmax(pre)
            if pre2 == y[i]:
                right+=1
        return right/len(X)

In [ ]:
%%time
lr = LogisticRegressionClassifier(max_iter=100)
lr.fit(x_train, y_train)
print(lr.score(x_val, y_val))

*********************************************************
round 0
 
err [ 0.02667144  0.12532341 -0.29941077  0.12086561  0.02655031] 
score 0.5476419325900295
*********************************************************
round 10
 
err [ 0.02827368  0.13476849 -0.26594263  0.09083187  0.01206859] 
score 0.5788158400615148
*********************************************************
round 20
 
err [ 0.02899782  0.13596321 -0.26272778  0.08755776  0.01020899] 
score 0.5810906061771114
